<b>This notebook is just a tutorial for you to get familiar with skip-gram and MapReduce.  
<font color="red">You don't need to hand in this notebook</font>, so feel free to jump to [Requirement section](#Assignment-Requirement) and directly work on your `mapper.py` and `reducer.py` if you already have the idea of how to do so.</b>  

# Week 05: Skip-gram and MapReduce

In previous assignments, you have known the concept of ngrams and how to generate them.  
This week, we are introducing another gram type, called *skip-gram*, to you.  
Also, we are going to calculate it on a large dataset, so you'll have to process it with the MapReduce technique.  

So, first thing first: what is skip-gram?  

## Skip-gram

<i>\[S\]kip-grams are a generalization of n-grams in which the components (typically words) need not be consecutive in the text under consideration, but may leave gaps that are skipped over.  - from [Wikipedia](https://en.wikipedia.org/wiki/N-gram#Skip-gram)</i>  

That is, skip-gram is actually the same as ngram, but allowed to skip some words in between.  
In the sentence <i>"Strong winds blew roofs away"</i>, two of its bigrams are <i>"winds blew"</i> and <i>"blew roofs"</i>, while <i>"blew away"</i> is one of the skipgrams with distance 2, since it skipped one word <i>"roofs"</i> .  
As you can see, skipgram is able to capture the phrase seperated by other words.  

Now consider another sentence

> "Skip-gram is used to predict the context word for a given target word".

With a pivot word *predict*, all of its skip-grams within distance 5 are as below:
```
.------------------------------------------------------------------------------.
| distance || -5 |     -4    | -3 |  -2  | -1 |  1  |    2    |  3   |  4  | 5 |
|----------||----|-----------|----|------|----|-----|---------|------|-----|---|
| predict  || -  | Skip-gram | is | used | to | the | context | word | for | a |
'------------------------------------------------------------------------------'
```

<a name="Practice"></a>
### Practice: Distance table of skip-gram

Now, let's practice!  
Given a sentence <i>"Skip-gram is used to predict the context word for a given target word"</i>, <u>**output all of its skip-gram with distance between -3 to 3</u> and show the result in a table**.  

**Example**
```
distance      -3            -2            -1            1             2             3             
--------------------------------------------------------------------------------------------
Skip-gram     -             -             -             is            used          to            
is            -             -             Skip-gram     used          to            predict       
used          -             Skip-gram     is            to            predict       the           
to            Skip-gram     is            used          predict       the           context       
predict       is            used          to            the           context       word          
the           used          to            predict       context       word          for           
context       to            predict       the           word          for           a             
word          predict       the           context       for           a             given         
for           the           context       word          a             given         target        
a             context       word          for           given         target        word          
given         word          for           a             target        word          -             
target        for           a             given         word          -             -             
word          a             given         target        -             -             -             
```

\*Hint: Try to get the skip-grams for a single word first if you have trouble generating them all at once. 
```
(predict, is, -3)
(predict, used, -2)
(predict, to, -1)
(predict, the, 1)
(predict, context, 2)
(predict, word, 3)
```

In [17]:
tokens = "Skip-gram is used to predict the context word for a given target word".split()
token_length = len(tokens)
dis_range = 3

dis_str = "distance".ljust(15)
for i in range(dis_range*2):
    if i-dis_range >=0:
        dis_str += str(i-dis_range+1).ljust(15)
    else:
        dis_str += str(i-dis_range).ljust(15)
print(dis_str)

for idx in range(token_length):
    tmp_str = str(tokens[idx]).ljust(15)
    for i in reversed(range(dis_range)):
        if idx-i-1 < 0:
            tmp_str += "-".ljust(15)
        else:
            tmp_str += str(tokens[idx-i-1]).ljust(15)
    for i in range(dis_range):
        if idx+i+1 >= token_length:
            tmp_str += "-".ljust(15)
        else:
            tmp_str += str(tokens[idx+i+1]).ljust(15)
    print(tmp_str)

distance       -3             -2             -1             1              2              3              
Skip-gram      -              -              -              is             used           to             
is             -              -              Skip-gram      used           to             predict        
used           -              Skip-gram      is             to             predict        the            
to             Skip-gram      is             used           predict        the            context        
predict        is             used           to             the            context        word           
the            used           to             predict        context        word           for            
context        to             predict        the            word           for            a              
word           predict        the            context        for            a              given          
for            the            context        w

## MapReduce

<i>MapReduce is a programming model and an associated implementation for processing and generating big data sets with a parallel, distributed algorithm on a cluster. - from [Wikipedia](https://en.wikipedia.org/wiki/MapReduce)</i> 

### Why MapReduce?

Imagine that you are working on a pretty large dataset, say all pages on Wikipedia (whose size has already reached 94GB in 2013).  
Most likely you are not able to process the whole corpus in the memory or on a single computer. Even a simple frequency counter would be challenging under such a huge data size.  
To deal with this, Google proposed a big-data processing model called MapReduce, and it has been implemented and supported by many distributed computing systems, such as Apache Hadoop.  
The core concept of MapReduce is to **split, apply and then combine**, so that each data segment can be handled separately.  

### Mapper-Shuffler-Reducer

![](https://www.todaysoftmag.com/images/articles/tsm33/large/a11.png)
<small><i> - image source: [Today Software Magazine](https://www.todaysoftmag.com/article/1358/hadoop-mapreduce-deep-diving-and-tuning)</i></small> 

As you can see in the picture:  
First, the whole data is split into some smaller partitions, each partition able to be processed by an independant machine.  
In this step, **mappers** will generate one or more key-value pair(s) that can easily be clustered.  
 - example: in a word counter, it would generate the word and the word's current count.  

Then, we will **shuffle** and group all outputs from mappers.  
 - example: sort the output from mappers.  

Lastly, we can combine the grouped values and **reduce** them into final results.  
 - example: calculate total frequency in each group.  

## MapReduce for skip-gram

Now, having the concepts of skip-gram and MapReduce in mind, it's time to put these all together: let's generate skip-gram table with MapReduce technique!  

It may sound scary to some of you, so let's break it down first.  
There are 3 steps to do, and each step is described as below:
1. **Mapper**: Print all skip-gram with its distance infomation, and the current count of it.  
   ```
   a b -3 1
   a c 3  1
   c e -2 1
   a c 1  1
   a b -3 1
   b d 2  1
   ```
2. **Shuffler**: Group all skipgrams by its text. This can be easily achieved with sorting.  
   ```
   a b -3 1
   a b -3 1
   a c 1  1
   a c 3  1
   b d 2  1
   c e -2 1
   ```
3. **Reducer** :  
   Since the results have been sorted in the previous step, we can easily calculate the frequency of each skip-gram with different distance.  
   So we can know that the frequency of skipgram `a b` with distance $-3$ should be $1+1=2$, while other skip-grams' are all $1$.

### Step 1: Mapper

First, in the mapper we want to generate all skip-grams within distance $-5$ to $5$.  
Remember that you've already done something similar in [previous Practice](#Practice)? Just modify it to MapReduce format!  

Output: 
 - `"{pivot}\t{word}\t{distance}\t{count}"`  

Example: 
```
predict is  -3  1
predict used    -2  1
predict the -1  1
predict the 1   1
...
```



In [54]:
import os
from string import punctuation

In [80]:
dis_range = 5
showlen = 10
cnt = 1

with open(os.path.join('data', 'wiki1G.txt'), encoding="utf-8") as f:
    for line in f:
        line = line.lower()
        Regtmp = line.split(" ")
        Regtmp = filter(None, Regtmp) # remove None from the List
        Reg = [i.strip(punctuation) for i in Regtmp]
        tokens = Reg
        token_length = len(tokens)
        for idx in range(token_length):
            for i in reversed(range(dis_range)):
                tmp_str = str(tokens[idx]) + "\t"
                if idx-i-1 < 0:
                    continue
                else:
                    tmp_str += str(tokens[idx-i-1]) + "\t" + str(-i-1) + "\t" + str(cnt) + "\n"
                    print(tmp_str)
            for i in range(dis_range):
                tmp_str = str(tokens[idx]) + "\t"
                if idx+i+1 >= token_length:
                    continue
                else:
                    tmp_str += str(tokens[idx+i+1]) + "\t" + str(i+1) + "\t" + str(cnt) + "\n"
                    print(tmp_str)
        
        break # for the sake of this practice, just test the first page now

anarchism	anarchism	1	1

anarchism	is	2	1

anarchism	a	3	1

anarchism	political	4	1

anarchism	philosophy	5	1

anarchism	anarchism	-1	1

anarchism	is	1	1

anarchism	a	2	1

anarchism	political	3	1

anarchism	philosophy	4	1

anarchism	and	5	1

is	anarchism	-2	1

is	anarchism	-1	1

is	a	1	1

is	political	2	1

is	philosophy	3	1

is	and	4	1

is	movement	5	1

a	anarchism	-3	1

a	anarchism	-2	1

a	is	-1	1

a	political	1	1

a	philosophy	2	1

a	and	3	1

a	movement	4	1

a	that	5	1

political	anarchism	-4	1

political	anarchism	-3	1

political	is	-2	1

political	a	-1	1

political	philosophy	1	1

political	and	2	1

political	movement	3	1

political	that	4	1

political	is	5	1

philosophy	anarchism	-5	1

philosophy	anarchism	-4	1

philosophy	is	-3	1

philosophy	a	-2	1

philosophy	political	-1	1

philosophy	and	1	1

philosophy	movement	2	1

philosophy	that	3	1

philosophy	is	4	1

philosophy	sceptical	5	1

and	anarchism	-5	1

and	is	-4	1

and	a	-3	1

and	political	-2	1

and	philosophy	-1	1

and	moveme


having	violent	3	1

having	turn	4	1

having	in	5	1

taken	down	-5	1

taken	authority	-4	1

taken	and	-3	1

taken	state	-2	1

taken	having	-1	1

taken	a	1	1

taken	violent	2	1

taken	turn	3	1

taken	in	4	1

taken	the	5	1

a	authority	-5	1

a	and	-4	1

a	state	-3	1

a	having	-2	1

a	taken	-1	1

a	violent	1	1

a	turn	2	1

a	in	3	1

a	the	4	1

a	past	5	1

violent	and	-5	1

violent	state	-4	1

violent	having	-3	1

violent	taken	-2	1

violent	a	-1	1

violent	turn	1	1

violent	in	2	1

violent	the	3	1

violent	past	4	1

violent	evolutionary	5	1

turn	state	-5	1

turn	having	-4	1

turn	taken	-3	1

turn	a	-2	1

turn	violent	-1	1

turn	in	1	1

turn	the	2	1

turn	past	3	1

turn	evolutionary	4	1

turn	tactics	5	1

in	having	-5	1

in	taken	-4	1

in	a	-3	1

in	violent	-2	1

in	turn	-1	1

in	the	1	1

in	past	2	1

in	evolutionary	3	1

in	tactics	4	1

in	aim	5	1

the	taken	-5	1

the	a	-4	1

the	violent	-3	1

the	turn	-2	1

the	in	-1	1

the	past	1	1

the	evolutionary	2	1

the	tactics	3	1

the	aim	4	1

t

been	synonymous	2	1

been	with	3	1

been	anarchism	4	1

been	its	5	1

largely	the	-5	1

largely	term	-4	1

largely	libertarian	-3	1

largely	has	-2	1

largely	been	-1	1

largely	synonymous	1	1

largely	with	2	1

largely	anarchism	3	1

largely	its	4	1

largely	meaning	5	1

synonymous	term	-5	1

synonymous	libertarian	-4	1

synonymous	has	-3	1

synonymous	been	-2	1

synonymous	largely	-1	1

synonymous	with	1	1

synonymous	anarchism	2	1

synonymous	its	3	1

synonymous	meaning	4	1

synonymous	has	5	1

with	libertarian	-5	1

with	has	-4	1

with	been	-3	1

with	largely	-2	1

with	synonymous	-1	1

with	anarchism	1	1

with	its	2	1

with	meaning	3	1

with	has	4	1

with	more	5	1

anarchism	has	-5	1

anarchism	been	-4	1

anarchism	largely	-3	1

anarchism	synonymous	-2	1

anarchism	with	-1	1

anarchism	its	1	1

anarchism	meaning	2	1

anarchism	has	3	1

anarchism	more	4	1

anarchism	recently	5	1

its	been	-5	1

its	largely	-4	1

its	synonymous	-3	1

its	with	-2	1

its	anarchism	-1	1

its	meaning	1	


to	in	2	1

to	or	3	1

to	progress	4	1

to	toward	5	1

exist	human	-5	1

exist	nature	-4	1

exist	allows	-3	1

exist	humans	-2	1

exist	to	-1	1

exist	in	1	1

exist	or	2	1

exist	progress	3	1

exist	toward	4	1

exist	such	5	1

in	nature	-5	1

in	allows	-4	1

in	humans	-3	1

in	to	-2	1

in	exist	-1	1

in	or	1	1

in	progress	2	1

in	toward	3	1

in	such	4	1

in	a	5	1

or	allows	-5	1

or	humans	-4	1

or	to	-3	1

or	exist	-2	1

or	in	-1	1

or	progress	1	1

or	toward	2	1

or	such	3	1

or	a	4	1

or	non-coercive	5	1

progress	humans	-5	1

progress	to	-4	1

progress	exist	-3	1

progress	in	-2	1

progress	or	-1	1

progress	toward	1	1

progress	such	2	1

progress	a	3	1

progress	non-coercive	4	1

progress	society	5	1

toward	to	-5	1

toward	exist	-4	1

toward	in	-3	1

toward	or	-2	1

toward	progress	-1	1

toward	such	1	1

toward	a	2	1

toward	non-coercive	3	1

toward	society	4	1

toward	and	5	1

such	exist	-5	1

such	in	-4	1

such	or	-3	1

such	progress	-2	1

such	toward	-1	1

such	a	1	1

such	no

taoist	philosophers	1	1

taoist	zhuang	2	1

taoist	zhou	3	1

taoist	and	4	1

taoist	laozi	5	1

philosophers	state	-5	1

philosophers	was	-4	1

philosophers	delineated	-3	1

philosophers	by	-2	1

philosophers	taoist	-1	1

philosophers	zhuang	1	1

philosophers	zhou	2	1

philosophers	and	3	1

philosophers	laozi	4	1

philosophers	alongside	5	1

zhuang	was	-5	1

zhuang	delineated	-4	1

zhuang	by	-3	1

zhuang	taoist	-2	1

zhuang	philosophers	-1	1

zhuang	zhou	1	1

zhuang	and	2	1

zhuang	laozi	3	1

zhuang	alongside	4	1

zhuang	stoicism	5	1

zhou	delineated	-5	1

zhou	by	-4	1

zhou	taoist	-3	1

zhou	philosophers	-2	1

zhou	zhuang	-1	1

zhou	and	1	1

zhou	laozi	2	1

zhou	alongside	3	1

zhou	stoicism	4	1

zhou	taoism	5	1

and	by	-5	1

and	taoist	-4	1

and	philosophers	-3	1

and	zhuang	-2	1

and	zhou	-1	1

and	laozi	1	1

and	alongside	2	1

and	stoicism	3	1

and	taoism	4	1

and	has	5	1

laozi	taoist	-5	1

laozi	philosophers	-4	1

laozi	zhuang	-3	1

laozi	zhou	-2	1

laozi	and	-1	1

laozi	alongside	

...


later	state	-4	1

later	and	-3	1

later	ferrer	-2	1

later	was	-1	1

later	arrested	1	1

later	nonetheless	2	1

later	his	3	1

later	ideas	4	1

later	formed	5	1

arrested	state	-5	1

arrested	and	-4	1

arrested	ferrer	-3	1

arrested	was	-2	1

arrested	later	-1	1

arrested	nonetheless	1	1

arrested	his	2	1

arrested	ideas	3	1

arrested	formed	4	1

arrested	the	5	1

nonetheless	and	-5	1

nonetheless	ferrer	-4	1

nonetheless	was	-3	1

nonetheless	later	-2	1

nonetheless	arrested	-1	1

nonetheless	his	1	1

nonetheless	ideas	2	1

nonetheless	formed	3	1

nonetheless	the	4	1

nonetheless	inspiration	5	1

his	ferrer	-5	1

his	was	-4	1

his	later	-3	1

his	arrested	-2	1

his	nonetheless	-1	1

his	ideas	1	1

his	formed	2	1

his	the	3	1

his	inspiration	4	1

his	for	5	1

ideas	was	-5	1

ideas	later	-4	1

ideas	arrested	-3	1

ideas	nonetheless	-2	1

ideas	his	-1	1

ideas	formed	1	1

ideas	the	2	1

ideas	inspiration	3	1

ideas	for	4	1

ideas	a	5	1

formed	later	-5	1

formed	arrested	-4	1

formed	n

instead	people	2	1

instead	being	3	1

instead	able	4	1

instead	to	5	1

of	of	-5	1

of	its	-4	1

of	political	-3	1

of	tendencies	-2	1

of	instead	-1	1

of	people	1	1

of	being	2	1

of	able	3	1

of	to	4	1

of	control	5	1

people	its	-5	1

people	political	-4	1

people	tendencies	-3	1

people	instead	-2	1

people	of	-1	1

people	being	1	1

people	able	2	1

people	to	3	1

people	control	4	1

people	the	5	1

being	political	-5	1

being	tendencies	-4	1

being	instead	-3	1

being	of	-2	1

being	people	-1	1

being	able	1	1

being	to	2	1

being	control	3	1

being	the	4	1

being	aspects	5	1

able	tendencies	-5	1

able	instead	-4	1

able	of	-3	1

able	people	-2	1

able	being	-1	1

able	to	1	1

able	control	2	1

able	the	3	1

able	aspects	4	1

able	of	5	1

to	instead	-5	1

to	of	-4	1

to	people	-3	1

to	being	-2	1

to	able	-1	1

to	control	1	1

to	the	2	1

to	aspects	3	1

to	of	4	1

to	their	5	1

control	of	-5	1

control	people	-4	1

control	being	-3	1

control	able	-2	1

control	to	-1	1

contr


for	used	2	1

for	art	3	1

for	as	4	1

for	a	5	1

or	life	-5	1

or	other	-4	1

or	anarchists	-3	1

or	advocated	-2	1

or	for	-1	1

or	used	1	1

or	art	2	1

or	as	3	1

or	a	4	1

or	means	5	1

used	other	-5	1

used	anarchists	-4	1

used	advocated	-3	1

used	for	-2	1

used	or	-1	1

used	art	1	1

used	as	2	1

used	a	3	1

used	means	4	1

used	to	5	1

art	anarchists	-5	1

art	advocated	-4	1

art	for	-3	1

art	or	-2	1

art	used	-1	1

art	as	1	1

art	a	2	1

art	means	3	1

art	to	4	1

art	achieve	5	1

as	advocated	-5	1

as	for	-4	1

as	or	-3	1

as	used	-2	1

as	art	-1	1

as	a	1	1

as	means	2	1

as	to	3	1

as	achieve	4	1

as	anarchist	5	1

a	for	-5	1

a	or	-4	1

a	used	-3	1

a	art	-2	1

a	as	-1	1

a	means	1	1

a	to	2	1

a	achieve	3	1

a	anarchist	4	1

a	ends	5	1

means	or	-5	1

means	used	-4	1

means	art	-3	1

means	as	-2	1

means	a	-1	1

means	to	1	1

means	achieve	2	1

means	anarchist	3	1

means	ends	4	1

means	in	5	1

to	used	-5	1

to	art	-4	1

to	as	-3	1

to	a	-2	1

to	means	-1	1

to	achiev

colin	anarchist	5	1

ward	entities	-5	1

ward	can	-4	1

ward	be	-3	1

ward	self-governing	-2	1

ward	colin	-1	1

ward	responds	1	1

ward	that	2	1

ward	major	3	1

ward	anarchist	4	1

ward	thinkers	5	1

responds	can	-5	1

responds	be	-4	1

responds	self-governing	-3	1

responds	colin	-2	1

responds	ward	-1	1

responds	that	1	1

responds	major	2	1

responds	anarchist	3	1

responds	thinkers	4	1

responds	advocated	5	1

that	be	-5	1

that	self-governing	-4	1

that	colin	-3	1

that	ward	-2	1

that	responds	-1	1

that	major	1	1

that	anarchist	2	1

that	thinkers	3	1

that	advocated	4	1

that	federalism	5	1

major	self-governing	-5	1

major	colin	-4	1

major	ward	-3	1

major	responds	-2	1

major	that	-1	1

major	anarchist	1	1

major	thinkers	2	1

major	advocated	3	1

major	federalism	4	1

major	philosophy	5	1

anarchist	colin	-5	1

anarchist	ward	-4	1

anarchist	responds	-3	1

anarchist	that	-2	1

anarchist	major	-1	1

anarchist	thinkers	1	1

anarchist	advocated	2	1

anarchist	federalism	3	1



ferguson	frances	-2	1

ferguson	l	-1	1

ferguson	joseph	1	1

ferguson	raz	2	1

ferguson	argues	3	1

ferguson	that	4	1

ferguson	the	5	1

joseph	review	-5	1

joseph	by	-4	1

joseph	frances	-3	1

joseph	l	-2	1

joseph	ferguson	-1	1

joseph	raz	1	1

joseph	argues	2	1

joseph	that	3	1

joseph	the	4	1

joseph	acceptance	5	1

raz	by	-5	1

raz	frances	-4	1

raz	l	-3	1

raz	ferguson	-2	1

raz	joseph	-1	1

raz	argues	1	1

raz	that	2	1

raz	the	3	1

raz	acceptance	4	1

raz	of	5	1

argues	frances	-5	1

argues	l	-4	1

argues	ferguson	-3	1

argues	joseph	-2	1

argues	raz	-1	1

argues	that	1	1

argues	the	2	1

argues	acceptance	3	1

argues	of	4	1

argues	authority	5	1

that	l	-5	1

that	ferguson	-4	1

that	joseph	-3	1

that	raz	-2	1

that	argues	-1	1

that	the	1	1

that	acceptance	2	1

that	of	3	1

that	authority	4	1

that	implies	5	1

the	ferguson	-5	1

the	joseph	-4	1

the	raz	-3	1

the	argues	-2	1

the	that	-1	1

the	acceptance	1	1

the	of	2	1

the	authority	3	1

the	implies	4	1

the	the	5	1

ac

### Step 2: Shuffler

All we need to do in the shuffler is sorting, so let's use the built-in command to do this for us!  

Try this on your terminal/command prompt ;)  
(You can get the sample input from [here](https://drive.google.com/drive/folders/1vKxr--sLd2J4kdsXUzJDBZdG3AmV4NGl?usp=sharing))

**Unix**  
```bash
sort -k1,3 < mapper.sample.tsv
```
**Windows**
```powershell
type mapper.sample.tsv | sort
```

### Step 3: Reducer

Since all the input should have been sorted in previous shuffler, the task of reducer is pretty simple: just count how many times the same gram appears, and then print the count out!

Input: 
 - `"{pivot}\t{word}\t{distance}\t{count}"`
 - You can get a sample input file `shuffler.sample.tsv` from [here](https://drive.google.com/drive/folders/1vKxr--sLd2J4kdsXUzJDBZdG3AmV4NGl?usp=sharing)

Output: 
 - `"{pivot}\t{word}\t{total_freq}\t{-5}\t{-4}\t{-3}\t{-2}\t{-1}\t{1}\t{2}\t{3}\t{4}\t{5}"`
 - The first two column are the skipgram; the third column is the sum of total frequency; column 4\~13 are the frequency with distance -5\~5, without 0.

Example:
 - `arouse  open    4       0       0       3       0       0       0       0       0       0       1`

Hints: 
1. Parse the input from shuffler
2. Check if this is the same skipgram as the previous one
3. If so, add the frequency according to its distance
4. If not, output the previous skipgram data

Note that you may NOT want to store all your counting results in a dict or any data structure.  
Recall that one purpose of MapReduce is to prevent memory exhaustion. It loses its value if you end up storing it again.  
Instead, <u>directly print it out or write it into a file</u> .  
(Don't get me wrong: of course you can store some temporary data, but let's not store the whole result and then print it out at once, okay?)


In [81]:
dis_range = 5
shlen = 10

with open(os.path.join('data', 'shuffler.sample.tsv')) as f:
    fw = open("reduce_test_ipynb.tsv", "w")
    tmp_test = 0 
    pivot_word = skip_word = ""
    pivot_cnt = 0
    pivot_list = [0] * shlen
    
    for line in f:
        # 1) Parse the input from shuffler
        # 2) Check if this is the same skipgram
        # 3) If so, add the frequency according to its distance
        # 4) If not, output the previous skipgram data
        line = line.replace("\n", "")
        Regtmp = line.split("\t")
        if len(pivot_word) == 0 and len(skip_word) == 0:
            pivot_word = Regtmp[0]
            skip_word = Regtmp[1]
               
        elif pivot_word == Regtmp[0] and skip_word == Regtmp[1]:
            a = 1
                
        else:
            tmp_print = pivot_word + "\t" + skip_word + "\t" + str(pivot_cnt) + "\t" 
            for i in range(shlen):
                tmp_print += str(pivot_list[i]) + "\t"
            if tmp_test < 30:
                print(tmp_print)
            fw.write(tmp_print+"\n")
            pivot_word = Regtmp[0]
            skip_word = Regtmp[1]
            pivot_cnt = 0
            pivot_list = [0] * shlen
            
        pivot_cnt += int(Regtmp[3])    
        if int(Regtmp[2]) < 0:
            pivot_list[int(Regtmp[2])+dis_range] += 1
        else:
            pivot_list[int(Regtmp[2])+dis_range-1] += 1
        
        tmp_test += 1
    fw.close()

1539	a	1	0	0	0	0	0	0	0	0	0	1	
1539	anarchisme	1	0	1	0	0	0	0	0	0	0	0	
1539	anarchy	1	0	0	0	1	0	0	0	0	0	0	
1539	and	1	0	0	1	0	0	0	0	0	0	0	
1539	as	1	1	0	0	0	0	0	0	0	0	0	
1539	early	1	0	0	0	0	0	1	0	0	0	0	
1539	empahised	1	0	0	0	0	0	0	0	0	1	0	
1539	english	1	0	0	0	0	0	0	1	0	0	0	
1539	from	1	0	0	0	0	1	0	0	0	0	0	
1539	usages	1	0	0	0	0	0	0	0	1	0	0	
1642	anarchism	1	1	0	0	0	0	0	0	0	0	0	
1642	anarchisme	1	0	0	0	0	0	0	1	0	0	0	
1642	anarchy	1	0	0	0	0	0	0	0	0	1	0	
1642	and	1	0	0	0	0	0	0	0	1	0	0	
1642	appears	1	0	1	0	0	0	0	0	0	0	0	
1642	as	1	0	0	0	0	0	1	0	0	0	0	
1642	english	1	0	0	0	1	0	0	0	0	0	0	
1642	from	2	0	0	0	0	1	0	0	0	0	1	
1642	in	1	0	0	1	0	0	0	0	0	0	0	
1756	1808	1	0	0	0	0	0	0	0	0	0	1	
1756	1836	1	0	0	0	0	0	1	0	0	0	0	
1756	and	1	0	0	0	0	0	0	1	0	0	0	
1756	as	1	0	0	1	0	0	0	0	0	0	0	
1756	century	1	1	0	0	0	0	0	0	0	0	0	
1756	godwin	1	0	0	0	0	1	0	0	0	0	0	
1756	such	1	0	1	0	0	0	0	0	0	0	0	
1756	weitling	1	0	0	0	0	0	0	0	0	1	0	
1756	wilhelm	1	0	0	0	0	0	0	0	1	0	0	


### Step 4: Combine them together!  

Now you can move your code above into mapper.py and reducer.py (with some tiny modifications, of course), and this is your assignment this week!   
See below for detailed requirement description.  

**Hints: What should I modify in my mapper and reducer?**  

1. Receive/pass data from standard I/O, rather than the file (We've already done this for you)
2. Process with the whole dataset, rather than only the first line

That's it!  

The processing takes some times (~1hr w/o parallel computing), so go enjoy some coffee or movies (or sleep) during the waiting time ;)

<a name="Assignment-Requirement"></a>
## Assignment Requirement 

1. You need to implement the `mapper.py` and `reducer.py` to calculate the skip-gram table.

2. In `mapper.py`, you need to generate skipgrams with distance within -5 to 5 (inclusive).  
   - Input: Pure text file (`wiki1G.txt`) with each line as a wikipage.
   - Output: `"{pivot}\t{word}\t{distance}\t{count}"`
   - Example: 
     ```
     predict is  -3  1
     predict used    -2  1
     predict the -1  1
     predict the 1   1
     ...
     ```
   - Sample output: `mapper.sample.tsv` (Find it [here](https://drive.google.com/drive/folders/1vKxr--sLd2J4kdsXUzJDBZdG3AmV4NGl?usp=sharing); no need to be exactly the same)

3. In `reducer.py`, you have to collect the output from the shuffler (`sort`) and generate the skip-gram table.
   - Input: `"{pivot}\t{word}\t{distance}\t{count}"`
   - Output: 
     - `"{pivot}\t{word}\t{total}\t{-5}\t{-4}\t{-3}\t{-2}\t{-1}\t{1}\t{2}\t{3}\t{4}\t{5}"`
     - The first two column are the skipgram; the third column is the sum of total frequency; column 4\~13 are the frequency with distance -5\~5, without 0.
   - Example:
     ```
     arouse  of      1       0       0       0       0       0       0       0       1       0       0
     arouse  open    4       0       0       3       0       0       0       0       0       0       1
     arouse  so      2       0       1       0       0       0       0       0       0       1       0
     arouse  sufficiently    1       0       1       0       0       0       0       0       0       0       0
     ...
     ```
   - Sample output: `reducer.sample.tsv` (Find it [here](https://drive.google.com/drive/folders/1vKxr--sLd2J4kdsXUzJDBZdG3AmV4NGl?usp=sharing); no need to be exactly the same)

4. Concate your MapReduce procedure and generate the skip-gram on wiki1G dataset
   - Unix: 
     - Use the [local map-reduce tool](https://github.com/dspp779/local-mapreduce) (faster),
     - or run it directly: `python mapper.py < wiki1G.txt | sort -k1,2 -k3n | python reducer.py > skipgram.tsv` (slower)
   - Windows: 
     - CMD: `python mapper.py < wiki1G.txt | sort | python reducer.py > skipgram.tsv`
     - PS: `type wiki1G.txt | python mapper.py | sort | python reducer.py > skipgram.tsv`
     - or the bash environment you installed last week.  
   - See [Appendix](#built-in-command) if you want to know what these commands mean

During the demo, you need to 

1. show us your skip-gram result on the given dataset, and
2. explain your implementation in `mapper.py` and `reducer.py`.  

Note that the final result would be a large file (~6 GB), so **you may want to show it with `more` or `less` command**.  

## TA's note

Congratulations! You've learned how to calculate skipgram frequency and to deal with a huge dataset with MapReduce technique.  

Remember to <b><a href="https://docs.google.com/spreadsheets/d/1QGeYl5dsD9sFO9SYg4DIKk-xr-yGjRDOOLKZqCLDv2E/edit?usp=sharing">make an appoiment with TA</a> to demo/explain your implementation <u>before <font color="red">10/21 15:30</font></u></b> .  
You should also submit your `mapper.py` and `reducer.py` to <a href="https://eeclass.nthu.edu.tw/course/homework/3285">eeclass</a> .

<a name="built-in-command"></a>
## Appendix: useful built-in commands

Several built-in commands are very useful in the MapReducer procedure.  
Here we introduce `cat` and `type`, `<` and `>`, `sort`, and pipe `|`.  

### cat (on Unix)
`cat` command, which is definitly not indicating some cute creatures (*meow~*), is the abbreviation of `concatenate`. ([doc](https://man7.org/linux/man-pages/man1/cat.1.html))   

When you `cat` a file, it means you want to print the content from a file (or some files) to standard output.  
Now open your bash and test the command below!  
```bash
cat file.txt
```

You should see something like this: 

![picture](https://i.imgur.com/Z9shOYQ.png)

### type (on Windows)
`type` command works exactly the same as `cat` on Unix, but without its cute nickname (Shame on you, Windows). ([doc](https://docs.microsoft.com/en-us/windows-server/administration/windows-commands/type))  

Similarly, if you `type` a file, it means you print the content from a file (or some files) to standard output.  

```powershell
type file.txt
```

You should see something like this:  
![](https://i.imgur.com/5WFhxkq.png)

### `>`? `<`? `>///<`? 

`<` and `>` are the I/O redirections.  
`program < filename` means that you want to redirect the input from a file to a program, while `program > filename` means that you want to redirect the output of a program to that file.  

For example,
```bash
echo "hello world" > greet.txt
```
writes the string "hello world" into a file `greet.txt`.  

On the other hand, 
```bash
head < greet.txt
```
makes `head` receive the content from `greet.txt`, so it will print out the string in `greet.txt`.  
![](https://i.imgur.com/swxv8LG.png)
<small>p.s. `>///<` is just a joke. Don't take it seriously.</small>

### sort

As its name suggests, `sort` sorts the data that it receives. (doc on [Linux](https://man7.org/linux/man-pages/man1/sort.1.html) and on [Windows](https://docs.microsoft.com/en-us/windows-server/administration/windows-commands/sort))  
Try this:
```
sort sample.txt
```
You can see that the content has been sorted before printed onto your screen.  
![](https://i.imgur.com/QFEq3Tc.png)

### Pipe `|`

Pipe passes the output from previous program to the next program.  
For example, 
```bash
python program.py | sort
```
will pass the output of `program.py` to `sort` command.  

In [11]:
import os
from string import punctuation
flag = 0
with open(os.path.join('data', 'skipgram.tsv'), encoding="utf-8") as f:
    tmp = 0
    for line in f:
        if (line[0] == "a"):
            flag = 1
        if flag == 1:
            print(line)
            tmp += 1
            if tmp > 10000:
                break;

a	0	3626	305	256	294	318	80	295	1043	351	420	264

a-0	0	1	0	0	0	0	0	0	0	0	0	1

a"=0	0	2	0	0	0	0	1	0	0	1	0	0

a"/0	0	1	0	0	0	0	0	0	0	1	0	0

a	00	32	4	2	4	4	1	3	1	5	7	1

a	0-0	5	0	1	0	0	0	2	2	0	0	0

a	0,0	3	0	1	0	1	0	0	0	0	1	0

a	0/0	2	1	0	1	0	0	0	0	0	0	0

a	0.0	27	2	3	4	7	0	2	0	2	6	1

a	0+0	1	0	0	0	0	0	0	0	0	0	1

a0	0	1	0	0	1	0	0	0	0	0	0	0

a	000	80	8	13	11	5	1	1	3	1	21	16

a	0,0,0	1	0	0	0	0	0	0	0	0	0	1

a	0:00	1	0	0	0	0	1	0	0	0	0	0

a	0.00	6	1	0	0	1	0	1	0	2	0	1

a	0000	13	3	1	1	3	1	0	1	1	1	1

a	0.000	3	1	1	0	0	0	0	0	1	0	0

a	0.0.0.0	3	1	0	0	0	0	0	0	0	1	1

a	00:00	3	1	1	0	0	0	0	0	0	0	1

a	00000	1	0	0	0	0	0	0	0	0	0	1

a	000000	1	1	0	0	0	0	0	0	0	0	0

a	00000000-0000-0000-0000-000000000000	1	0	1	0	0	0	0	0	0	0	0

a	0.000000065	2	0	1	0	0	0	0	1	0	0	0

a	0.00000026	1	0	1	0	0	0	0	0	0	0	0

a	0.0000005	1	1	0	0	0	0	0	0	0	0	0

a000:0000	address	1	0	0	0	1	0	0	0	0	0	0

a000:0000	and	1	0	0	0	0	0	1	0	0	0	0

a000:0000	b000:ffff	1	0	0	0	0	0	0	1	0	0	0

a000:0000	in	1	0	0	0	0	0	0	0	1	0	0

a000:0000	mode	1	

a	1.0016	1	0	0	0	0	0	0	0	0	0	1

a	100180	1	0	0	1	0	0	0	0	0	0	0

a	100,182	2	0	0	1	0	0	0	0	1	0	0

a	100,189	1	1	0	0	0	0	0	0	0	0	0

a	10,019	1	0	1	0	0	0	0	0	0	0	0

a-100	1959	1	0	1	0	0	0	0	0	0	0	0

a100	1963	1	0	0	0	1	0	0	0	0	0	0

a-100	1965	1	0	0	0	0	0	0	0	1	0	0

a1001	a1057	1	0	0	1	0	0	0	0	0	0	0

a1001	and	1	0	0	0	1	0	0	0	0	0	0

a1001	bishops	1	0	0	0	0	0	0	0	1	0	0

a1001	bridge	1	0	0	0	0	0	0	0	0	0	1

a1001	business	1	0	0	0	0	0	0	0	0	0	1

a1001	each	1	0	1	0	0	0	0	0	0	0	0

a1001	near	1	0	0	0	0	0	0	1	0	0	0

a1001	of	1	1	0	0	0	0	0	0	0	0	0

a1001	on	2	0	0	0	1	0	1	0	0	0	0

a1001	other	1	0	0	1	0	0	0	0	0	0	0

a	1,001-piano	1	0	0	0	0	0	1	0	0	0	0

a1001	polaroid	1	0	0	0	0	0	1	0	0	0	0

a1001	railway	1	0	0	0	0	0	0	0	0	1	0

a1001	square	1	0	0	0	0	0	0	0	0	1	0

a1001	the	5	0	1	0	0	2	0	1	1	0	0

a1001	to	1	1	0	0	0	0	0	0	0	0	0

a	1002	18	1	4	4	3	1	0	0	3	2	0

a	1,002	4	2	0	0	0	0	0	0	1	1	0

a	1.002	1	0	1	0	0	0	0	0	0	0	0

a	10:02	1	0	0	0	1	0	0	0	0	0	0

a	10.02	2	0	0	0	0	0	0	0	2	0	0

a	100.2	4	0	0	0	0	0	0	0


a	109,371	1	0	0	0	0	0	0	0	0	1	0

a	109,375	2	2	0	0	0	0	0	0	0	0	0

a	109,376	2	0	0	1	0	0	0	0	1	0	0

a	1094	9	1	2	2	0	1	0	0	1	0	2

a	1,094	5	1	0	0	0	0	0	0	4	0	0

a	109.4	1	1	0	0	0	0	0	0	0	0	0

a	10,941	1	0	0	0	0	0	0	0	1	0	0

a	109,410	2	0	0	1	0	0	0	0	1	0	0

a	10,942	1	0	0	0	0	0	0	0	1	0	0

a	109,424	1	0	0	0	0	0	0	0	0	1	0

a	109,428	2	0	0	1	0	0	0	0	1	0	0

a	109.443782	1	0	0	0	1	0	0	0	0	0	0

a	109,459	2	0	0	1	0	0	0	0	1	0	0

a	10,947	1	0	0	0	0	0	0	0	0	0	1

a	109,472	1	0	0	0	0	0	0	0	0	0	1

a	1094/95	1	0	0	0	0	1	0	0	0	0	0

a	109,496	2	0	0	1	0	0	0	0	1	0	0

a1094	b1122	1	0	0	0	0	0	0	0	1	0	0

a1094	benhall	1	0	0	1	0	0	0	0	0	0	0

a1094	by	1	0	0	0	1	0	0	0	0	0	0

a1094	in	1	0	1	0	0	0	0	0	0	0	0

a1094	leads	1	0	0	0	0	0	0	0	0	1	0

a	109.4-meter	1	0	0	0	0	0	0	1	0	0	0

a1094	road	1	0	0	0	0	0	1	0	0	0	0

a	1,094-seat	1	0	0	0	0	0	1	0	0	0	0

a1094	street	1	1	0	0	0	0	0	0	0	0	0

a1094	the	2	0	0	0	0	1	0	1	0	0	0

a1094	to	1	0	0	0	0	0	0	0	0	0	1

a	1095	27	3	7	5	3	4	0	0	1	2	2

a	1,095	2	0	1	0	0	0	0	0	1	0	0

a	1:

a1152	near	1	0	0	0	0	0	1	0	0	0	0

a1152	off	1	0	0	0	1	0	0	0	0	0	0

a1152	raf	1	0	0	0	0	0	0	0	0	1	0

a1152	rendlesham	1	0	1	0	0	0	0	0	0	0	0

a1152	the	2	0	0	0	0	1	0	1	0	0	0

a	1153	25	4	5	4	7	2	0	0	3	0	0

a	1,153	5	0	1	0	0	0	0	0	4	0	0

a	11.53	1	1	0	0	0	0	0	0	0	0	0

a	115.3	1	0	0	0	0	0	0	1	0	0	0

a	11530	3	1	0	0	0	1	0	0	0	1	0

a	115,331	1	0	0	0	0	0	0	0	1	0	0

a	115,341	1	1	0	0	0	0	0	0	0	0	0

a	115,382	1	1	0	0	0	0	0	0	0	0	0

a	115,385	2	0	0	1	0	0	0	0	1	0	0

a	1154	29	6	3	3	6	4	0	0	3	4	0

a	1,154	4	0	1	1	1	0	0	0	1	0	0

a	1.1.5.4	2	0	0	0	0	1	0	0	0	1	0

a	11,541	1	0	0	0	0	0	0	0	1	0	0

a	1,154,116	1	0	0	0	0	0	0	0	1	0	0

a	115,439	2	0	0	1	0	0	0	0	1	0	0

a	11,544	1	0	0	1	0	0	0	0	0	0	0

a	11,546	1	0	1	0	0	0	0	0	0	0	0

a	1155	21	3	3	6	5	0	0	0	0	2	2

a	1,155	5	1	0	1	0	0	0	0	2	0	1

a	11550	1	0	0	0	0	0	0	0	1	0	0

a	115,500,000	1	0	0	0	0	0	1	0	0	0	0

a	115,513	1	0	0	1	0	0	0	0	0	0	0

a	11553	2	0	0	0	1	0	0	0	1	0	0

a	11,553	1	0	0	1	0	0	0	0	0	0	0

a	11,553,427	1	0	0	0	0	0	0	0	1	0	0

a	115,541	2	0	0	1	0